# Circular Movement Simulation
In this notebook we will make use of the calibration phantom and simulate the first movement, the circular beam around it.

We will make use of our phantom as well as make different types of reconstruction (for starters we will use the SIRT) and evaluate the best methods and showcase the results with svg videos.

In [ ]:
import tomosipo as ts
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# ------------------------------------------- Constants ------------------------------------------ #
PHANTOM_PATH = 'C:/Users/wilru/Documents/LU/S4/Thesis/simulation/phantoms/'
PHANTOM_PIXELS = 512


### Geometry

First step is to create the geometry, in which we will use the parallel beam geometry, adding a range of 360 angles, to make 360 different slices to our circular movement.

In [ ]:
# Create volume
'''
    Creates a volume.
    - The volume is a three-dimensional unit cube
        and is composed of 32 voxels in each dimension.
'''
vg = ts.volume(shape=(PHANTOM_PIXELS, PHANTOM_PIXELS, PHANTOM_PIXELS), size=(1, 1, 1))
# print(vg)

# Create circular parallel beam projection geometry
'''
    Creates a circular parallel-beam projection geometry.
    - The parallel-beam geometry has 32 angles that are
        equi-spaced over a half arc.
    - The detector has 48 pixels in each dimension and
        1.5 units high and wide.
'''
pg = ts.parallel(angles=360, shape=(PHANTOM_PIXELS, PHANTOM_PIXELS), size=(1.5, 1.5))
# print(pg)

# Display volume and geometry as an svg animation
svg = ts.svg(vg, pg)
svg

### Projection
Secondly, we create the projection usign the projector operator A and a set of data x.
In here we will load our corresponding phantom (calibration phantom), previously generated and saved.

In [ ]:
# CLoad projection data ---> calibration phantom
x = np.load(PHANTOM_PATH + 'calibration_phantom.npy')
x = np.load("../phantoms/foam_phantom.npy")

In [ ]:
# In order to make sure the phantom is correct we will print some slices
x_t = x.transpose(2, 0, 1)
fig = plt.figure(figsize=(8, 8))
plt.subplot(2, 2, 1)
plt.title("Slice 2")
plt.imshow(x[2, :, :], cmap='gray')
plt.subplot(2, 2, 2)
plt.title(f"Transposed - Slice {PHANTOM_PIXELS // 4}")
plt.imshow(x_t[PHANTOM_PIXELS // 4, :, :], cmap='gray')
plt.subplot(2, 2, 3)
plt.title(f"Transposed - Slice {PHANTOM_PIXELS // 2}")
plt.imshow(x_t[PHANTOM_PIXELS // 2, :, :], cmap='gray')
plt.subplot(2, 2, 4)
plt.title(f"Transposed - Slice {PHANTOM_PIXELS * 3 // 4}")
plt.imshow(x_t[(3 * PHANTOM_PIXELS) // 4, :, :], cmap='gray')
plt.show()

In [ ]:
# We can now create a projection operator A
'''
    The projection operator (A) has two useful properties:
    - domain_shape
    - range_shape
    that can be used to create volume and projection data

    The projection data (x) is stored as a stack of sinograms
    (following the ASTRA-toolbox convention):
    - First dimension -> the height of the detector
    - Second dimension -> the number of angles
    - Third dimension -> the width of the detector
'''
A = ts.operator(vg, pg)
print(f"Domain A shape: {A.domain_shape}, Domain A range: {A.range_shape}")

# Create a projection by applying the operator to the data x
y = A(x)
print(f"Projection shape: {y.shape}")

# Plot the projections using matplotlib
fig = plt.figure(figsize=(8, 8))
plt.subplot(2, 2, 1)
plt.title("First projection (0/32)")
plt.imshow(y[:, 0, :])
plt.subplot(2, 2, 2)
plt.title(f"Quarter rotation (8/32 = 1/4)")
plt.imshow(y[:, 8, :])
plt.subplot(2, 2, 3)
plt.title("Half rotation (16/32)")
plt.imshow(y[:, 16, :])
plt.subplot(2, 2, 4)
plt.title(f"Three quarters rotation (24/32)")
plt.imshow(y[:, 24, :])
plt.show();


### Reconstruction - SIRT
We will now as the last step, the reconstruction.
In this case we will use the simple SIRT reconstruction method, and observe how we are ablw to reconstruct our object.

In [ ]:
# Prepare matrices R and C, we clamp values using np.mimum because there are values divided by 0
R = 1 / A(np.ones(A.domain_shape, dtype=np.float32))
R = np.minimum(R, 1 / ts.epsilon)
C = 1 / A.T(np.ones(A.range_shape, dtype=np.float32))
C = np.minimum(C, 1 / ts.epsilon)

# Reconstruction frmo the sincogram stack y into x_rec
num_iterations = 100
x_rec = np.zeros(A.domain_shape, dtype=np.float32)

for i in range(num_iterations):
    x_rec += C * A.T(R * (y - A(x_rec)))

In [ ]:
# Plot the reconstruction and phantom comparison
x_rec_t = x_rec.transpose(2, 0, 1)
fig = plt.figure(figsize=(8, 8))
plt.subplot(2, 2, 1)
plt.title("Central slice of phantom")
plt.imshow(x[16, :, :], cmap='gray')
plt.subplot(2, 2, 2)
plt.title(f"Central slice of reconstruction")
plt.imshow(x_rec[16, :, :], cmap='gray')
plt.subplot(2, 2, 3)
plt.title(f"Transposed - Phantom Slice {PHANTOM_PIXELS // 2}")
plt.imshow(x_t[PHANTOM_PIXELS // 2, :, :], cmap='gray')
plt.subplot(2, 2, 4)
plt.title(f"Transposed - Reconstruction Slice {PHANTOM_PIXELS * 3 // 4}")
plt.imshow(x_rec_t[PHANTOM_PIXELS // 2, :, :], cmap='gray')
plt.show();
